In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import explained_variance_score, mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense,Flatten
from keras.layers.convolutional import Conv1D,MaxPooling1D
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam,RMSprop,Adadelta,Adagrad,Adamax,Nadam
from keras.models import load_model
import seaborn as sns

In [ ]:
def series_to_supervised(data_np,n_lags,n_outs):
    samples = len(data_np)-(n_lags+n_outs-1)
    Inputs = []
    Targets = np.empty((samples,n_outs))
    for i in range(samples):
        step = i+n_lags
        sample = data_np[i:step]
        Inputs.append(sample)
        label = data_np[step:(step+n_outs)]
        j = 0
        for a in label:
            Targets[i,j] = a
            j = j+1
    return np.array(Inputs),Targets

In [ ]:
def plot_model_history(model_summary):
    plt.plot(model_summary.history['loss'])
    plt.plot(model_summary.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train','test'],loc='upper right')
    plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/m.tech project solar/weather_2017+2018_hourly.csv")
data_as_np = dataset.GHI.values

In [ ]:
# fit a model
def create_model(input_shape,n_outs,train_x,train_y,test_x,test_y,rate):

    # define model
    model = Sequential()
    model.add(Conv1D(100, 3, activation='relu', input_shape=input_shape))
    model.add(Conv1D(64, 3, activation='relu'))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(n_outs))

    cp1 = ModelCheckpoint('CNN/', save_best_only = True)
    model.compile(loss='mse', optimizer= adam(learning_rate=rate), metrics=[RootMeanSquaredError()])
    model_summary = model.fit(train_x, train_y, validation_data = (test_x,test_y), epochs = 80, batch_size = 64, callbacks=[cp1])
    plot_model_history(model_summary)

    model = load_model('CNN/')

    return model

In [ ]:
data_np = data_as_np.reshape(-1,1)
n_lags = 480
n_out = 24

# Prepare data
Inputs,Targets = series_to_supervised(data_np,n_lags,n_out)

Inputs_model, Targets_model = Inputs[:-10], Targets[:-10]
Inputs_plot, Targets_plot = Inputs[-10:], Targets[-10:]

no_samples = Targets_model.shape[0]
split = int(0.3 * no_samples)

features = 1
n_input = n_lags * features

# Splitting data into train and test data set
Inputs_model = Inputs_model.reshape((no_samples,n_input))

train_x,train_y = Inputs_model[:-split],Targets_model[:-split]
test_x,test_y = Inputs_model[-split:],Targets_model[-split:]

In [ ]:
def rmse_error(pred,y):
  return np.sqrt(mean_squared_error(pred,y))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# New Section

In [ ]:
# predicted values
predicted = model.predict(Inputs_plot)
actual = Targets_plot

In [ ]:
# Next 1 day plotting
comp_df = pd.DataFrame({"actual":actual[0].flatten(),"predicted":predicted[0].flatten()})
sns.lineplot(data = comp_df)

In [ ]:
# Next 2 day prediction
comp_df = pd.DataFrame({"actual":actual[0:2].flatten(),"predicted":predicted[0:2].flatten()})
sns.lineplot(data = comp_df)

In [ ]:
# Next 7 day prediction
comp_df = pd.DataFrame({"actual":actual[0:7].flatten(),"predicted":predicted[0:7].flatten()})
sns.lineplot(data = comp_df)

In [ ]:
train_pred, test_pred = model.predict(train_x),model.predict(test_x)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
# Performance matrix on training set
{'explained_variance_score':r2_score(train_pred,train_y),
      'mean_squared_error':mean_squared_error(train_pred,train_y),
          'mean_absolute_error':mean_absolute_error(train_pred,train_y)
}


In [ ]:
#Performance matrix on test set
{'explained_variance_score':r2_score(test_pred,test_y),
    'mean_squared_error':mean_squared_error(test_pred,test_y),
          'mean_absolute_error':mean_absolute_error(test_pred,test_y)
}
